<h2>US Domestic Flight Analysis</h2>

@author: Laxmi Prajapat

In [1]:
# import relevant libraries
import pandas as pd
import numpy as np
import datetime

In [2]:
# general functions

# merging two dataframes via a left join
def left_join(df1, df2, key1, key2):
    joined = df1.merge(df2, how = 'left', left_on = key1, right_on = key2)
    del joined[key2]
    return joined

# function for type casting columns
# col_type = str, float, int...
# else assume date column
def type_cast(df, col_list, col_type = None, time_format = "%Y-%m-%d"):
    if len(col_list) == 0:
        raise ValueError('Length of column list must be > 0')
    if len(col_list) == 1:
        col_list = col_list[0]
    if col_type:
        df[col_list] = df[col_list].astype(col_type)
    else:
        df[col_list] = pd.to_datetime(df[col_list], format = time_format)
    return df

In [3]:
# read in files
#data = pd.read_csv('data-flights_medium.csv', header = 0)
data = pd.read_csv('data-flights_large.csv', header = 0)
weather = pd.read_csv('data-weather.csv', header = 0)

# shape of data
print data.shape
print weather.shape

(10000000, 19)
(35488695, 6)


<h4>External data sources</h4>

In [4]:
# external data downloaded

### 1) state abbreviations
# https://github.com/jasonong/List-of-US-States/blob/master/states.csv
states = pd.read_csv('states.csv', header = 0)
# append "PR" = "Puerto Rico" and "VI" = "United States Virgin Islands"
# [i for i in data["Departure_State"].unique().tolist() if i not in states.index.values.tolist()]
states = states.append({"Abbreviation": "PR", "State": "Puerto Rico"},ignore_index = True)
states = states.append({"Abbreviation": "VI", "State": "United States Virgin Islands"},ignore_index = True)
print states.shape


### 2) airport codes (IATA) - airport names - location
# https://github.com/datasets/airport-codes
airport_codes = pd.read_csv('airport-codes.csv', header = 0)
print airport_codes.shape
# take subset for US and where IATA code is not missing
airport_codes = airport_codes[airport_codes["iso_country"] == "US"]
airport_codes = airport_codes.dropna(subset = ["iata_code"])
# IATA codes should be unique, but are not in this case as the code can be re-used for closed/re-opened airports

# print airport_codes.shape
# print airport_codes["iata_code"].nunique()

# drop the rows where duplicates directly impact our flights dataset so that we have 1:1 mapping
# airport = AUS and SEA, drop the rows representing closed airport
airport_codes = airport_codes.drop([11129, 35250])

# split coordinates column into longitude and latitude
# round to 2 d.p to match accuracy of flights data
airport_codes['longitude'], airport_codes['latitude'] = airport_codes['coordinates'].str.split(', ', 1).str
airport_codes[['longitude', 'latitude']] = airport_codes[['longitude', 'latitude']].astype(float)
airport_codes = airport_codes.round({'longitude': 2, 'latitude': 2})

# add in information for PSE, SOP, STX, SJU, airports as these were missing in a join attempt
# http://www.airnav.com
airport_codes = airport_codes.append({"longitude": -66.56, "latitude": 18.01, "iata_code": "PSE", 
                                      "name": "Mercedita Airport"}, ignore_index = True)
airport_codes = airport_codes.append({"longitude": -79.39, "latitude": 35.24, "iata_code": "SOP", 
                                      "name": "Moore County Airport"}, ignore_index = True)
airport_codes = airport_codes.append({"iata_code": "STX", "name": "Henry E Rohlsen Airport"}, ignore_index = True)
airport_codes = airport_codes.append({"iata_code": "SJU", "name": "Luis Munoz Marin International Airport"}, 
                                     ignore_index = True)
airport_codes = airport_codes.append({"iata_code": "STT", "name": "Cyril E King Airport"}, ignore_index = True)

# quick retrieval of information
def search_airport(code):
    print airport_codes[airport_codes["iata_code"] == code][["iso_region","iata_code","name", "longitude", "latitude"]]
    

### 3) airline names
# https://openflights.org/data.html
airlines = pd.read_csv('airline-id.csv', header = 0)
airlines['Code'] = airlines['Code'].astype(str)
# subset containing codes corresponding to our flights dataset
data["Airline_code"] = data["Airline_code"].astype(str)
airlines = airlines[airlines["Code"].isin(data["Airline_code"].unique().tolist())]
print airlines.shape


### 4) state-region mapping
# https://github.com/cphalpert/census-regions
regions = pd.read_csv('us census bureau regions and divisions.csv', header = 0)
print regions.shape
regions = regions.append({"State": "Puerto Rico", "State Code": "PR", "Region": "Not Applicable", "Division": "Not Applicable"}, ignore_index = True)
regions = regions.append({"State": "United States Virgin Islands", "State Code": "VI", "Region": "Not Applicable", "Division": "Not Applicable"}, ignore_index = True)

(53, 2)
(46235, 12)
(23, 3)
(51, 4)


<h4>Data cleaning and initial profiling</h4>

In [5]:
# columns present
print data.columns.values.tolist()
print weather.columns.values.tolist()

['Date', 'Airline', 'Airline_code', 'Arrival_airport', 'Arrival_state', 'Departure_Airport', 'Departure_State', 'Departure_actual', 'Departure_delay', 'Arrival_actual', 'Arrival_delay', 'Arrival_schedule', 'Departure_schedule', 'DC_Longitude', 'DC_Latitude', 'Longitude', 'Latitude', 'Route', 'Path_order']
['Date', 'airport', 'time', 'temperature', 'snow', 'wind']


In [6]:
# observe first 50 rows of flight data
data.head(50)

,Date,Airline,Airline_code,Arrival_airport,Arrival_state,Departure_Airport,Departure_State,Departure_actual,Departure_delay,Arrival_actual,Arrival_delay,Arrival_schedule,Departure_schedule,DC_Longitude,DC_Latitude,Longitude,Latitude,Route,Path_order
0,2010-12-27,HA,19690,SAN,CA,HNL,HI,1311,1,2047,12,2035.0,1310.0,-157.92,21.31,-157.92,21.31,HI to CA,1
1,2004-04-10,HA,19690,SAN,CA,HNL,HI,1336,11,2146,11,2135.0,1325.0,-157.92,21.31,-157.92,21.31,HI to CA,1
2,2004-04-29,HA,19690,SAN,CA,HNL,HI,1404,39,2223,48,2135.0,1325.0,-157.92,21.31,-157.92,21.31,HI to CA,1
3,2008-09-09,HA,19690,SAN,CA,HNL,HI,1333,-2,2147,-3,2150.0,1335.0,-157.92,21.31,-157.92,21.31,HI to CA,1
4,2011-03-30,HA,19690,SAN,CA,HNL,HI,1314,-1,2158,23,2135.0,1315.0,-157.92,21.31,-157.92,21.31,HI to CA,1
5,2011-03-24,HA,19690,SAN,CA,HNL,HI,1314,-1,2140,5,2135.0,1315.0,-157.92,21.31,-157.92,21.31,HI to CA,1
6,2008-12-29,HA,19690,SAN,CA,HNL,HI,1416,6,2131,1,2130.0,1410.0,-157.92,21.31,-157.92,21.31,HI to CA,1
7,2012-01-29,HA,19690,SAN,CA,HNL,HI,1501,111,2238,123,2035.0,1310.0,-157.92,21.31,-157.92,21.31,HI to CA,1
8,2011-06-15,HA,19690,SAN,CA,HNL,HI,1303,3,2133,3,2130.0,1300.0,-157.92,21.31,-157.92,21.31,HI to CA,1
9,2012-05-29,HA,19690,SAN,CA,HNL,HI,1345,0,2212,2,2210.0,1345.0,-157.92,21.31,-157.92,21.31,HI to CA,1


In [7]:
# observe first 50 rows of weather data
weather.head(50)

,Date,airport,time,temperature,snow,wind
0,2006-03-14,ORD,2041,35,NaN,18.6
1,2006-09-08,SLC,807,72,NaN,6.5
2,2006-10-22,MCO,1202,81,0.0,3.1
3,2006-10-22,MCO,602,81,0.0,3.1
4,2007-07-09,MRY,1011,64,NaN,0.7
5,2006-08-01,IAH,1256,85,NaN,5.1
6,2006-01-22,MDW,1330,27,3.0,0.9
7,2007-07-04,DFW,1259,80,NaN,5.7
8,2007-07-04,DFW,930,80,NaN,5.7
9,2006-08-17,SEA,644,63,NaN,2.8


In [8]:
# current data types
print data.dtypes
print weather.dtypes

# segregating string and float columns for weather dataset
data_str_cols = ['Airline', 'Airline_code', 'Arrival_airport', 'Arrival_state', 'Departure_Airport', 
                 'Departure_State', 'Route', 'Path_order']
data_float_cols = ['Departure_actual', 'Departure_delay', 'Arrival_actual', 'Arrival_delay', 'Arrival_schedule', 
                   'Departure_schedule', 'DC_Longitude', 'DC_Latitude', 'Longitude', 'Latitude']

# segregating string and float columns for weather dataset
weather_str_cols = ['airport']
weather_float_cols = ['time', 'temperature', 'snow', 'wind']

# same date column for both datasets
date_cols = ['Date']
date_format = "%Y-%m-%d"

data = type_cast(data, data_str_cols, str)
data = type_cast(data, data_float_cols, float)
data = type_cast(data, date_cols)

weather = type_cast(weather, weather_str_cols, str)
weather = type_cast(weather, weather_float_cols, float)
weather = type_cast(weather, date_cols)

# observe changes
print data.dtypes
print weather.dtypes

# create additional date columns
data["Year"] = data["Date"].dt.year
data["Month"] = data["Date"].dt.month
data["Day"] = data["Date"].dt.day

weather["Year"] = weather["Date"].dt.year
weather["Month"] = weather["Date"].dt.month
weather["Day"] = weather["Date"].dt.day

Date                   object
Airline                object
Airline_code           object
Arrival_airport        object
Arrival_state          object
Departure_Airport      object
Departure_State        object
Departure_actual        int64
Departure_delay         int64
Arrival_actual          int64
Arrival_delay           int64
Arrival_schedule      float64
Departure_schedule    float64
DC_Longitude          float64
DC_Latitude           float64
Longitude             float64
Latitude              float64
Route                  object
Path_order              int64
dtype: object
Date            object
airport         object
time             int64
temperature      int64
snow           float64
wind           float64
dtype: object
Date                  datetime64[ns]
Airline                       object
Airline_code                  object
Arrival_airport               object
Arrival_state                 object
Departure_Airport             object
Departure_State               object
Depar

In [9]:
# basic descriptive statistics for numerical columns
print data.describe()
print weather.describe()

       Departure_actual  Departure_delay  Arrival_actual  Arrival_delay  \
count      1.000000e+07     1.000000e+07    1.000000e+07   1.000000e+07   
mean       1.339896e+03     7.977523e+00    1.485823e+03   5.590595e+00   
std        4.749350e+02     2.989364e+01    4.960542e+02   3.307119e+01   
min        1.000000e+00    -1.200000e+03    1.000000e+00  -3.360000e+02   
25%        9.310000e+02    -4.000000e+00    1.105000e+03  -1.000000e+01   
50%        1.330000e+03    -1.000000e+00    1.514000e+03  -2.000000e+00   
75%        1.731000e+03     6.000000e+00    1.908000e+03   1.000000e+01   
max        2.400000e+03     1.382000e+03    2.400000e+03   1.435000e+03   

       Arrival_schedule  Departure_schedule  DC_Longitude   DC_Latitude  \
count      9.999999e+06        9.999999e+06  1.000000e+07  1.000000e+07   
mean       1.495351e+03        1.332818e+03 -9.481311e+01  3.709718e+01   
std        4.781425e+02        4.636574e+02  1.669296e+01  5.832348e+00   
min        0.000000e+00 

In [10]:
# time period range
print data["Date"].min(), data["Date"].max()
print weather["Date"].min(), weather["Date"].max()

2002-01-01 00:00:00 2012-12-31 00:00:00
2005-01-01 00:00:00 2007-07-18 00:00:00


In [11]:
# initial insights on composition of data

# data:
# unique airlines/airline codes covered = 23
print "Unique airlines/codes: ",data["Airline"].nunique(), data["Airline_code"].nunique()

# unique departure/arrival airports = 234, 67 respectively
print "Unique departure airports: ",data["Departure_Airport"].nunique()
print "Unique arrival airports: ",data["Arrival_airport"].nunique()

# overall distinct airports covered = 255
print "Unique airports: ",len(set(data["Departure_Airport"].unique().tolist() + data["Arrival_airport"].unique().tolist()))

# unique departure/arrival states = 52, 32 respectively
print "Unique departure states: ",data["Departure_State"].nunique()
print "Unique arrival states: ",data["Arrival_state"].nunique()

# overall distinct states covered = 52
print "Unique states: ",len(set(data["Departure_State"].unique().tolist() + data["Arrival_state"].unique().tolist()))

# unique routes
print "Unique routes: ",data["Route"].nunique()
print "Most popular route: ",data['Route'].value_counts().index[0]

# year with most flights
print "Year with most flights: ",data["Year"].value_counts().index[0]

# airline with most flights
print "Airline with most flights: ",data["Airline"].value_counts().index[0]

# weather:
# overall distinct airports
print "Unique airports: ",weather["airport"].nunique()

Unique airlines/codes:  23 23
Unique departure airports:  234
Unique arrival airports:  67
Unique airports:  255
Unique departure states:  52
Unique arrival states:  32
Unique states:  52
Unique routes:  586
Most popular route:  CA to CA
Year with most flights:  2005
Airline with most flights:  US
Unique airports:  273


In [12]:
# check to see if there are any duplicates
# there are 15 duplicates in the flight data across all variables
print data.duplicated().sum()

# there are 9266064 duplicates in the weather data across all variables
print weather.duplicated().sum()

# drop duplicate rows in the datasets and compare shapes
s1 = data.shape[0]
data = data.drop_duplicates()
s2 = data.shape[0]
print "Before: {}, After: {}".format(s1,s2)

s1 = weather.shape[0]
weather = weather.drop_duplicates()
s2 = weather.shape[0]
print "Before: {}, After: {}".format(s1,s2)

15
9266064
Before: 10000000, After: 9999985
Before: 35488695, After: 26222631


In [13]:
# check if there are missing values
# there is only 1 for "Arrival_schedule" and "Departure_schedule"
print data.isnull().sum()

# several missing values for snow and wind
print weather.isnull().sum()

# total number of rows containing missing values
print "Missing rows for data df: ",data.shape[0] - data.dropna().shape[0]
print "Missing rows in weather df: ",weather.shape[0] - weather.dropna().shape[0]

Date                  0
Airline               0
Airline_code          0
Arrival_airport       0
Arrival_state         0
Departure_Airport     0
Departure_State       0
Departure_actual      0
Departure_delay       0
Arrival_actual        0
Arrival_delay         0
Arrival_schedule      1
Departure_schedule    1
DC_Longitude          0
DC_Latitude           0
Longitude             0
Latitude              0
Route                 0
Path_order            0
Year                  0
Month                 0
Day                   0
dtype: int64
Date                  0
airport               0
time                  0
temperature           0
snow           18742261
wind               6486
Year                  0
Month                 0
Day                   0
dtype: int64
Missing rows for data df:  1
Missing rows in weather df:  18742905


In [14]:
# impute missing values in flights data
# Arrival_actual - Arrival_delay = Arrival_schedule
# Departure_actual - Departure_delay = Departure_schedule

# general method
"""
def impute_schedule(schedule, actual, delay):
    if pd.isnull(schedule):
        return actual - delay
    else:
        return

data["Arrival_schedule"] = data[["Arrival_schedule", "Arrival_actual", "Arrival_delay"]].apply(lambda x: impute_schedule(x[0],x[1],x[2]), axis = 1)
data["Departure_schedule"] = data[["Departure_schedule", "Departure_actual", "Departure_delay"]].apply(lambda x: impute_schedule(x[0],x[1],x[2]), axis = 1)
"""

# fast method using known index for missing value
missing_index = data[pd.isnull(data['Arrival_schedule'])].index.values[0]
data["Arrival_schedule"].loc[missing_index] = data["Arrival_actual"].loc[missing_index] - data["Arrival_delay"].loc[missing_index]
data["Departure_schedule"].loc[missing_index] = data["Departure_actual"].loc[missing_index] - data["Departure_delay"].loc[missing_index]

In [15]:
# airports where we have missing values for wind
print weather[pd.isnull(weather['snow'])]['airport'].unique()

# airports where we have missing values for wind
print weather[pd.isnull(weather['wind'])]['airport'].unique()

# snow and wind slightly negatively correlated with temperature (as expected)
print weather.corr()

['ORD' 'SLC' 'MRY' 'IAH' 'DFW' 'SEA' 'MEM' 'BOS' 'DEN' 'LAX' 'PBI' 'FLL'
 'PHX' 'RDM' 'ATL' 'RIC' 'ABQ' 'BDL' 'BZN' 'SJC' 'DCA' 'LAS' 'IAD' 'MLB'
 'GPT' 'SMF' 'PWM' 'SFO' 'ALB' 'TUL' 'ELP' 'JFK' 'LYH' 'MSP' 'MCO' 'CLT'
 'BGR' 'AUS' 'FAT' 'GNV' 'RDU' 'DTW' 'SAN' 'SDF' 'CAE' 'MDT' 'BUR' 'MSY'
 'MFR' 'CHO' 'SAV' 'PDX' 'MCI' 'HNL' 'OAK' 'CLL' 'HSV' 'BTR' 'ROC' 'LIT'
 'RSW' 'GSP' 'MHT' 'ACV' 'ONT' 'ORF' 'AZO' 'SNA' 'PVD' 'SRQ' 'DAL' 'BNA'
 'FAY' 'PIT' 'HOU' 'LGB' 'MIA' 'RNO' 'EVV' 'GRR' 'HPN' 'RAP' 'ASE' 'IND'
 'PHL' 'SGF' 'JAX' 'HTS' 'MDW' 'BOI' 'TXK' 'TOL' 'GRB' 'IDA' 'OGG' 'TUS'
 'SBA' 'PSP' 'TLH' 'LEX' 'BHM' 'GSO' 'LGA' 'ILM' 'MLI' 'MAF' 'SJU' 'CRW'
 'JAN' 'XNA' 'COS' 'LAN' 'CID' 'LWS' 'MCN' 'ANC' 'LNK' 'PHF' 'TYS' 'LIH'
 'GJT' 'TUP' 'PNS' 'EWR' 'PSC' 'LFT' 'ITO' 'HRL' 'BWI' 'GRK' 'CHA' 'MGM'
 'CRP' 'ABY' 'FAI' 'ABE' 'EYW' 'CDC' 'BRO' 'DSM' 'BFL' 'MSN' 'EUG' 'DHN'
 'OKC' 'IPL' 'KOA' 'TYR' 'MOD' 'CAK' 'GEG' 'SJT' 'DAY' 'LBB' 'HLN' 'TPA'
 'PFN' 'PIH' 'SBP' 'DAB' 'ACT' 'AVL' 'TWF' 'MSO' 'D

In [16]:
# take subset of weather for 1 year - arbitrarily chose 2006 - 10.2 million rows now
"""
weather = weather[weather["Year"] == 2006]
print weather.shape
weather.head()
"""

'\nweather = weather[weather["Year"] == 2006]\nprint weather.shape\nweather.head()\n'

In [17]:
# distribution of missing values across months - very evenly spread
print weather[pd.isnull(weather["snow"])]["Month"].value_counts()

1     1902123
5     1881848
3     1875629
6     1858862
4     1795717
7     1679191
2     1670153
8     1295434
11    1218539
10    1199419
9     1195935
12    1169411
Name: Month, dtype: int64


In [ ]:
# imputing missing values for "snow" - 18,742,261 missing (71.5% of dataset)
# UPDATE: now 7221183 missing for 2006 subset

# subset of weather dataset where there was snow fall
snow_fall = weather[weather['snow'] > 0]
print snow_fall.shape

# assuming the snow pattern is similar year on year for simplicity (could distinguish years if wanted to)
# group by airport and month, and find max temperature where there was snow and average snow fall
max_snow_temp = snow_fall.groupby(['airport', 'Month']).agg({'temperature': max, 'snow': np.mean}).reset_index()
print max_snow_temp.head()

### LOGIC:
# in the weather data, if there is a missing value present for snow fall...
# for the corresponding airport/month of year if the temperature is beyond the max temp where there was snow fall...
# safely assume there is no snow fall
# otherwise if the temperature is lower, then take the average snow fall amount
# and if there is no data for snow fall for a given airport in a given month, then assume there is no snow fall
# assuming that no other external factors influence snow fall - ideal scenario

# very computationally expensive

def snow_impute(airport, month, temperature, snow):
    if pd.isnull(snow):
        subset = max_snow_temp[(max_snow_temp['airport'] == airport) & (max_snow_temp['Month'] == month)]
        if subset.shape[0] > 0:
            if temperature > subset['temperature'].values[0]:
                return 0.0
            else:
                return subset['snow'].values[0]
        else:
            return 0.0
    else:
        return snow
    
weather['snow_imputed'] = weather[['airport', 'Month', 'temperature', 'snow']].apply(lambda x: snow_impute(x[0],x[1],x[2],x[3]), axis=1)

# observe missing values after imputing
print weather.isnull().sum()

In [18]:
# imputing missing values for "wind" - 6486 missing (0.025% of dataset)
# these values will be imputed with zero as it only represents a small portion of the dataset
# and an aggregation will be perfomed at a daily level which should absorb the discrepancies

def wind_impute(x):
    if pd.isnull(x):
        return 0.0
    else:
        return x

# a more computationally expensive method would be to impute the missing value with e.g. the min/mean wind speed
# for that particular day

weather['wind'] = weather['wind'].apply(lambda x: wind_impute(x))

# check that missing values accounted for
print weather.isnull().sum()

Date                  0
airport               0
time                  0
temperature           0
snow           18742261
wind                  0
Year                  0
Month                 0
Day                   0
dtype: int64


In [19]:
# group up weather at a daily level
# weather_grouped = weather.groupby(["airport", "Year", "Month", "Day"]).agg({"snow": "mean", "temperature": "mean"}).reset_index()

weather_grouped = weather.groupby(["airport", "Date"]).agg({"wind": "mean", "temperature": "mean"}).reset_index()

weather_grouped.head()

,airport,Date,temperature,wind
0,ABE,2005-01-01,48.0,4.3
1,ABE,2005-01-02,34.0,1.7
2,ABE,2005-01-03,44.0,0.6
3,ABE,2005-01-04,44.0,2.3
4,ABE,2005-01-05,37.0,6.8


In [20]:
weather_grouped.shape

(227711, 4)

In [21]:
# frequency of flight routes
print data['Route'].value_counts()

# determine if there were flights that landed in states that deviate from the recorded route
data['Route_from'], data['Route_to'] = data['Route'].str.split(' to ', 1).str
# the arrival state in "Route" does indeed match "Arrival_state"
print data['Route_to'].equals(data['Arrival_state'])

CA to CA    384121
CA to AZ    358621
TX to TX    318147
GA to FL    202044
FL to NC    147301
CA to CO    140405
TX to AZ    137059
TX to LA    128440
TX to CO    123026
FL to PA    120084
FL to GA    115160
TX to OK    111582
AK to AK    103577
TX to GA    102878
WA to CA     96211
VA to NC     90287
PA to NC     88646
IL to PA     83621
NC to GA     80372
TX to FL     80311
GA to GA     77986
TX to CA     77679
TX to NC     76046
NV to CA     75000
FL to KY     71003
CO to CA     66449
WA to AK     66445
AZ to CA     65541
GA to PA     64629
UT to AZ     64535
             ...  
FL to MO        12
PR to KY        11
OH to OK        10
LA to LA        10
AL to LA        10
MN to ME        10
NH to MA         8
CA to MO         5
FL to LA         5
SC to LA         4
MT to AZ         4
MS to FL         4
RI to MA         3
NE to IA         3
NC to LA         2
AR to LA         2
AL to FL         2
ID to NV         2
AL to MS         2
IN to MI         1
LA to FL         1
WV to GA    

In [22]:
# possible values and frequencies for path_order
data["Path_order"].value_counts()

1    9686071
2     313914
Name: Path_order, dtype: int64

<h4>Appending data from external sources</h4>

In [23]:
# use state mapping df for departure/arrival US state abbreviations
# data["Arrival_State_Name"] = data["Arrival_state"].apply(lambda x: states.loc[str(x)].values[0])
# data["Departure_State_Name"] = data["Departure_State"].apply(lambda x: states.loc[str(x)].values[0])

# faster than the above
data = left_join(data, states, "Arrival_state", "Abbreviation")
data = data.rename(columns={'State': 'Arrival_State_Name'})

data = left_join(data, states, "Departure_State", "Abbreviation")
data = data.rename(columns={'State': 'Departure_State_Name'})

In [24]:
# checking if the Longitude and Latitude columns are different to DC_Longitude and DC_Latitude
# 'False' in both cases
print data['DC_Longitude'].equals(data['Longitude'])
print data['DC_Latitude'].equals(data['Latitude'])

# checking rows where DC_Longitude and DC_Latitude and not equal to Longitude and Latitude
print data[data['DC_Longitude'] != data['Longitude']]
# these are rows where Path_order = 2 is specified, however only 313914 have path_order = 2

# therefore, use DC_Longitude and DC_Latitude as the departure airport coordinates
data = data.rename(columns={'DC_Longitude': 'Departure_Longitude', 'DC_Latitude': 'Departure_Latitude'})
del data['Longitude']
del data['Latitude']

# delete path_order = 2 rows, as it is not a complete view, and can be re-computed later
del data['Path_order']
s1 = data.shape[0]
data = data.drop_duplicates()
s2 = data.shape[0]
print "Before: {}, After: {}".format(s1,s2)
# now we should have 1 row per 1 flight

# use the airport_codes dataset to determine long/lat for arrival airports
data = left_join(data, airport_codes[['iata_code', 'longitude', 'latitude', 'name']], 'Arrival_airport', 'iata_code')
data = data.rename(columns={'name': 'Arrival_Airport_Name', 'latitude': 'Arrival_Latitude', 'longitude': 'Arrival_Longitude'})

# retrieve airport name for departure airport as well
data = left_join(data, airport_codes[['iata_code', 'name']], 'Departure_Airport', 'iata_code')
data = data.rename(columns={'name': 'Departure_Airport_Name'})

# checks:
# search_airport('HNL')
# search_airport('SAN')

False
False
              Date Airline Airline_code Arrival_airport Arrival_state  \
15542   2003-12-28      OO        20304             SJT            TX   
15543   2004-06-17      OO        20304             SJT            TX   
15544   2004-06-28      XE        20374             SJT            TX   
15545   2004-06-23      XE        20374             SJT            TX   
15546   2004-06-08      OO        20304             SJT            TX   
15547   2005-05-17      OO        20304             SJT            TX   
15548   2005-05-25      OO        20304             SJT            TX   
15549   2005-05-26      OO        20304             SJT            TX   
15550   2003-11-28      OO        20304             SJT            TX   
15551   2003-11-19      OO        20304             SJT            TX   
15552   2004-07-24      OO        20304             SJT            TX   
15553   2004-05-24      OO        20304             SJT            TX   
15554   2004-07-01      OO        20304

In [25]:
# retrieve airline names using airlines dataframe
data = left_join(data, airlines[['Code', 'Name']], 'Airline_code', 'Code')
data = data.rename(columns={'Name': 'Airline_Name'})

In [26]:
# retrieve region names using regions dataframe
data = left_join(data, regions[['State Code', 'Region']], 'Arrival_state', 'State Code')
data = data.rename(columns={'Region': 'Arrival_Region'})

data = left_join(data, regions[['State Code', 'Region']], 'Departure_State', 'State Code')
data = data.rename(columns={'Region': 'Departure_Region'})

In [27]:
# check again if there are missing values
print data.isnull().sum()

# there are 5851 missing arrival airport names and corresponding long/lat - for "PSE" and "SOP"
# there are 32700 missing departure airport names - STX, SJU and STT
# UPDATE: these have been added to the airport_codes/states dataframe
# print data[pd.isnull(data["Arrival_Airport_Name"])]["Arrival_airport"].unique().tolist()
# print data[pd.isnull(data["Departure_Airport_Name"])]["Departure_Airport"].unique()

Date                      0
Airline                   0
Airline_code              0
Arrival_airport           0
Arrival_state             0
Departure_Airport         0
Departure_State           0
Departure_actual          0
Departure_delay           0
Arrival_actual            0
Arrival_delay             0
Arrival_schedule          0
Departure_schedule        0
Departure_Longitude       0
Departure_Latitude        0
Route                     0
Year                      0
Month                     0
Day                       0
Route_from                0
Route_to                  0
Arrival_State_Name        0
Departure_State_Name      0
Arrival_Longitude         0
Arrival_Latitude          0
Arrival_Airport_Name      0
Departure_Airport_Name    0
Airline_Name              0
Arrival_Region            0
Departure_Region          0
dtype: int64


In [ ]:
# sort values - computationally expensive (therefore not necessary)
data = data.sort_values(['Date', 'Departure_actual'])
weather = weather.sort_values(['Date', 'time'])

<h4>Additional feature engineering</h4>

In [28]:
# create binary flag columns for flight delay types, as well as overall delay time
data["Departure_Delayed"] = data["Departure_delay"].apply(lambda x: 1 if x > 0 else 0)
data["Arrival_Delayed"] = data["Arrival_delay"].apply(lambda x: 1 if x > 0 else 0)
data["Overall_Delay_Time"] = data[["Departure_delay", "Arrival_delay"]].apply(lambda x: x[0]+x[1], axis = 1)
data["Overall_Flight_Delayed"] = data[["Departure_delay", "Arrival_delay"]].apply(lambda x: 1 if x[0]+x[1] > 0 else 0, axis = 1)

In [29]:
data.head()

,Date,Airline,Airline_code,Arrival_airport,Arrival_state,Departure_Airport,Departure_State,Departure_actual,Departure_delay,Arrival_actual,...,Arrival_Latitude,Arrival_Airport_Name,Departure_Airport_Name,Airline_Name,Arrival_Region,Departure_Region,Departure_Delayed,Arrival_Delayed,Overall_Delay_Time,Overall_Flight_Delayed
0,2010-12-27,HA,19690,SAN,CA,HNL,HI,1311.0,1.0,2047.0,...,32.73,San Diego International Airport,Honolulu International Airport,Hawaiian Airlines Inc.,West,West,1,1,13.0,1
1,2004-04-10,HA,19690,SAN,CA,HNL,HI,1336.0,11.0,2146.0,...,32.73,San Diego International Airport,Honolulu International Airport,Hawaiian Airlines Inc.,West,West,1,1,22.0,1
2,2004-04-29,HA,19690,SAN,CA,HNL,HI,1404.0,39.0,2223.0,...,32.73,San Diego International Airport,Honolulu International Airport,Hawaiian Airlines Inc.,West,West,1,1,87.0,1
3,2008-09-09,HA,19690,SAN,CA,HNL,HI,1333.0,-2.0,2147.0,...,32.73,San Diego International Airport,Honolulu International Airport,Hawaiian Airlines Inc.,West,West,0,0,-5.0,0
4,2011-03-30,HA,19690,SAN,CA,HNL,HI,1314.0,-1.0,2158.0,...,32.73,San Diego International Airport,Honolulu International Airport,Hawaiian Airlines Inc.,West,West,0,1,22.0,1


In [ ]:
data.to_csv('data-flights-final.csv', index = False)

In [ ]:
weather_grouped.to_csv('weather-data-daily-final.csv', index = False)

<h3>NLP analysis on airling ratings</h3>

In [30]:
# import relevant libraries for NLP analysis
import re
import sklearn
import nltk
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer, word_tokenize
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer

In [31]:
relevant_airlines = data[["Airline_Name", "Airline"]].drop_duplicates().sort_values("Airline_Name")
relevant_airlines

,Airline_Name,Airline
1671310,ATA Airlines,TZ
43329,AirTran Airways Corporation,FL
12,Alaska Airlines Inc.,AS
69940,America West Airlines Inc.,HP
84411,American Airlines Inc.,AA
2425,Comair Inc.,OH
62043,Continental Air Lines Inc.,CO
43459,Delta Air Lines Inc.,DL
81014,Endeavor Air Inc.,9E
15575,Envoy Air,MQ


In [32]:
# this data was manually filtered for relevant airlines (as there was no key/code)
reviews = pd.read_csv('airline_ratings_subset.csv', header = 0)
print reviews.shape

(4283, 4)


In [33]:
reviews.head()

,airline_name,date,content,overall_rating
0,alaska-airlines,26/07/15,Checked into Alaska Airlines at the Santa Rosa...,1.0
1,alaska-airlines,23/07/15,"Short one and half hour flight, seats were com...",8.0
2,alaska-airlines,15/07/15,Flew down to Louisville from Anchorage with a ...,9.0
3,alaska-airlines,12/07/15,"One way, PDX-LAX. Everything was smooth sailin...",7.0
4,alaska-airlines,11/07/15,LAX-SEA on the 737-800. Flew on 4th of July an...,10.0


In [34]:
# impute missing ratings with zero
print reviews.isnull().sum()

reviews["overall_rating"] = reviews["overall_rating"].apply(lambda x: 0.0 if pd.isnull(x) else x)

print reviews.isnull().sum()

airline_name        0
date                0
content             0
overall_rating    603
dtype: int64
airline_name      0
date              0
content           0
overall_rating    0
dtype: int64


In [35]:
# typecast
reviews["date"] = pd.to_datetime(reviews["date"], format = "%d/%m/%y")
reviews["month"] = reviews["date"].dt.month
reviews["year"] = reviews["date"].dt.year
reviews["day"] = reviews["date"].dt.day

# for ease of use in Tableau
reviews["month_year"] = reviews[["month", "year"]].apply(lambda x: "1-"+str(x[0])+"-"+str(x[1]), axis = 1)
reviews

,airline_name,date,content,overall_rating,month,year,day,month_year
0,alaska-airlines,2015-07-26,Checked into Alaska Airlines at the Santa Rosa...,1.0,7,2015,26,1-7-2015
1,alaska-airlines,2015-07-23,"Short one and half hour flight, seats were com...",8.0,7,2015,23,1-7-2015
2,alaska-airlines,2015-07-15,Flew down to Louisville from Anchorage with a ...,9.0,7,2015,15,1-7-2015
3,alaska-airlines,2015-07-12,"One way, PDX-LAX. Everything was smooth sailin...",7.0,7,2015,12,1-7-2015
4,alaska-airlines,2015-07-11,LAX-SEA on the 737-800. Flew on 4th of July an...,10.0,7,2015,11,1-7-2015
5,alaska-airlines,2015-06-21,Just returned to the Greatland having flown wh...,9.0,6,2015,21,1-6-2015
6,alaska-airlines,2015-05-18,Returned recently from a trip to SEA from ANC ...,9.0,5,2015,18,1-5-2015
7,alaska-airlines,2015-05-16,Flight 893 9 May. I don't fly Alaska at all as...,10.0,5,2015,16,1-5-2015
8,alaska-airlines,2015-05-11,Flight AS838 on 6 May. Flew Alaska from Honolu...,9.0,5,2015,11,1-5-2015
9,alaska-airlines,2015-04-27,Flew with family SJC-Honolulu then returned to...,8.0,4,2015,27,1-4-2015


In [36]:
reviews.dtypes

airline_name              object
date              datetime64[ns]
content                   object
overall_rating           float64
month                      int64
year                       int64
day                        int64
month_year                object
dtype: object

In [37]:
# there are reviews for 13 of the 23 airlines we are concerned with
print reviews["airline_name"].nunique()
print reviews["airline_name"].value_counts()

13
united-airlines         840
american-airlines       612
delta-air-lines         466
us-airways              425
frontier-airlines       417
southwest-airlines      379
jetblue-airways         237
alaska-airlines         236
hawaiian-airlines       234
virgin-america          225
continental-airlines    156
skywest-airlines         35
comair                   21
Name: airline_name, dtype: int64


In [38]:
# dictionary that maps airline names from the reviews file to an airline abbreviation
review_airline_dict = {
    "united-airlines": "UA",
    "american-airlines": "AA",
    "delta-air-lines": "DL",
    "us-airways": "US",
    "frontier-airlines": "F9",
    "southwest-airlines": "WN",
    "jetblue-airways": "B6",
    "alaska-airlines": "AS",
    "hawaiian-airlines": "HA",
    "virgin-america": "VX",
    "continental-airlines": "CO",
    "skywest-airlines": "OO",
    "comair": "OH"
}

# obtain airline name and abbreviation
reviews["airline"] = reviews["airline_name"].apply(lambda x: review_airline_dict[x])
reviews = left_join(reviews, relevant_airlines, 'airline', 'Airline')

<h4>Cleaning of "content"</h4>

In [39]:
### dealing with negation words and incorrect spelling
negation_dictionary = {
    "not_happy ": "unhappy ",
    "not_happi ": "unhappi ",
    "not_explain ": "unexplain ",
    "not_explained ": "unexplained ",
    "not_understand ": "misunderstand ",
    "not_satisfied ": "dissatisfied ",
    "not_satisfi ": "dissatisfied ",
    "not_satisfy ": "dissatisfied ",
    "not_correct ": "incorrect ",
    "not_possible ": "impossible ",
    "not_excellent ": "unexcellent ",
    "not_good ": "bad ",
    "not_available ": "unavailable ",
    "not_told ": "nottold ",
    "not_tell ": "nottold ",
    "not_informed ": "uninformed ",
    "not_inform ": "uninformed ",
    "wasnt happy ": "unhappy ",
    "isnt happy ": "unhappy ",
    "un_happy ": "unhappy ",
    "wasnt_happy ": "unhappy ",
    "isnt_happy ": "unhappy "
}

# map negation words
def negation(x, dictionary):
    negation_list = ["not ", "isnt ", "isn't", "wasnt ", "wasn't ", " un "]
    negations = "|".join(negation_list)
    x = re.sub(negations, "not_", x)
    x = reduce(lambda x, y: x.replace(y, dictionary[y]), dictionary, x)
    x = re.sub("cannot_", "cannot ", x)
    x = re.sub("not_", "not ", x)
    return x


### stop word list and extensions
stop_words = list(set(stopwords.words('english')))
print len(stop_words)

# remove words from stop_words that could represent negation/negativity
remove_list = ["no", "haven", "needn", "shan", "doesn", "ain", "don", "weren", "wouldn", "aren", "mightn", "wasn",
              "couldn", "shouldn", "mustn", "hasn", "not", "didn", "won"]
for word in remove_list:
    stop_words.remove(word)

print len(stop_words)

# extend the stop word list to remove common words expected in flight reviews
add_list = ["mr", "mrs", "said", "i", "fly", "st", "flight", "still", "via", "travel", "get", "us", "airport"]
stop_words.extend(add_list)


### remove additional strings from text such as states, airports, cities
states_abbv_list = states["Abbreviation"].unique().tolist()
airline_abbv_list = relevant_airlines["Airline"].unique().tolist()
airport_abbv_list = list(set(data["Departure_Airport"].unique().tolist()+data["Arrival_airport"].unique().tolist()))
all_abbv = states_abbv_list+airline_abbv_list+airport_abbv_list
all_abbv_list = ['{0} '.format(elem) for elem in all_abbv]

# ...after lower conversion
states_list = states["State"].str.lower().unique().tolist()
airline_list = relevant_airlines["Airline_Name"].str.lower().unique().tolist()
airport_list = list(set(data["Departure_Airport_Name"].str.lower().unique().tolist()+data["Arrival_Airport_Name"].unique().tolist()))
city_list = ["new york", "boston", "philadelphia", "austin", "san francisco", "seattle", "chicago", 
             "los angeles", "washington", "louisville", "anchorage", "honolulu", "oakland"]
all_full = states_list+airline_list+airport_list+city_list
all_full_list = ['{0} '.format(elem) for elem in all_full]

def remove_abbrev(x):
    exclusions = "|".join(all_abbv_list)
    x = re.sub(exclusions, '', x)
    return x

def remove_full(x):
    exclusions = "|".join(all_full_list)
    x = re.sub(exclusions, '', x.lower())
    return x


### remove numbers and characters
def remove_char(x):
    pattern = re.compile("[^a-zA-Z ]")
    x = re.sub(pattern, " ", x)
    return x


### remove trailing spaces
def remove_space(x):
    x = x.rstrip()
    return x


### lemmatization
# only lemmatize verbs (default = nouns)
def lem(x):
    lem = WordNetLemmatizer()
    tokens = word_tokenize(x)
    lemma = ' '.join(lem.lemmatize(word, 'v') for word in tokens)
    return lemma


### stop word removal
def stop_word(x):
    tokens = word_tokenize(unicode(x, errors = "ignore"))
    stop = ' '.join(word for word in tokens if not word in stop_words)
    return stop

153
134


In [40]:
# cleaning review "content" text
def process_text(df, col_name):
    df["cleaned_text"] = df[col_name]
    df["cleaned_text"] = df["cleaned_text"].apply(lambda x: remove_char(x))
    df["cleaned_text"] = df["cleaned_text"].apply(lambda x: remove_abbrev(x))
    df["cleaned_text"] = df["cleaned_text"].apply(lambda x: remove_full(x))
    df["cleaned_text"] = df["cleaned_text"].apply(lambda x: stop_word(x))
    df["cleaned_text"] = df["cleaned_text"].apply(lambda x: negation(x, negation_dictionary))
    df["cleaned_text"] = df["cleaned_text"].apply(lambda x: remove_space(x))
    df["cleaned_text"] = df["cleaned_text"].apply(lambda x: lem(x))
    return df

reviews = process_text(reviews, "content")

In [41]:
reviews

,airline_name,date,content,overall_rating,month,year,day,month_year,airline,Airline_Name,cleaned_text
0,alaska-airlines,2015-07-26,Checked into Alaska Airlines at the Santa Rosa...,1.0,7,2015,26,1-7-2015,AS,Alaska Airlines Inc.,check airlines santa rosa hours take plane min...
1,alaska-airlines,2015-07-23,"Short one and half hour flight, seats were com...",8.0,7,2015,23,1-7-2015,AS,Alaska Airlines Inc.,short one half hour seat comfortable friendly ...
2,alaska-airlines,2015-07-15,Flew down to Louisville from Anchorage with a ...,9.0,7,2015,15,1-7-2015,AS,Alaska Airlines Inc.,fly stop way bump first nice crew great plane ...
3,alaska-airlines,2015-07-12,"One way, PDX-LAX. Everything was smooth sailin...",7.0,7,2015,12,1-7-2015,AS,Alaska Airlines Inc.,one way everything smooth sail seat comfortabl...
4,alaska-airlines,2015-07-11,LAX-SEA on the 737-800. Flew on 4th of July an...,10.0,7,2015,11,1-7-2015,AS,Alaska Airlines Inc.,fly th july atmosphere board festive friendly ...
5,alaska-airlines,2015-06-21,Just returned to the Greatland having flown wh...,9.0,6,2015,21,1-6-2015,AS,Alaska Airlines Inc.,return greatland fly favorite airline depart a...
6,alaska-airlines,2015-05-18,Returned recently from a trip to SEA from ANC ...,9.0,5,2015,18,1-5-2015,AS,Alaska Airlines Inc.,return recently trip eagle onto flight well ru...
7,alaska-airlines,2015-05-16,Flight 893 9 May. I don't fly Alaska at all as...,10.0,5,2015,16,1-5-2015,AS,Alaska Airlines Inc.,may don loyalties another airline present freq...
8,alaska-airlines,2015-05-11,Flight AS838 on 6 May. Flew Alaska from Honolu...,9.0,5,2015,11,1-5-2015,AS,Alaska Airlines Inc.,may fly may not expect much cost first class t...
9,alaska-airlines,2015-04-27,Flew with family SJC-Honolulu then returned to...,8.0,4,2015,27,1-4-2015,AS,Alaska Airlines Inc.,fly family return economy class plan variety s...


<h4>Sentiment analysis</h4>

In [42]:
# compute polarity and assign sentiment classification
def sentiment(pol):
    if pol > 0.0:
        return "positive"
    elif pol < 0.0:
        return "negative"
    else:
        return "neutral"
    
    
reviews["polarity"] = reviews["cleaned_text"].apply(lambda x: float(TextBlob(str(x)).sentiment.polarity))
reviews["sentiment_class"] = reviews["polarity"].apply(lambda x: sentiment(x))

In [43]:
# group up reviews and calculate average polarity on a monthly level
#monthly = reviews.groupby(['airline', 'Airline_Name', 'year', 'month']).agg({'polarity': 'mean', 'overall_rating': 'mean'}).reset_index()
monthly = reviews.groupby(['airline', 'Airline_Name', 'month_year']).agg({'polarity': 'mean', 'overall_rating': 'mean'}).reset_index()

In [44]:
monthly.head()

,airline,Airline_Name,month_year,polarity,overall_rating
0,AA,American Airlines Inc.,1-1-2014,0.083165,4.317073
1,AA,American Airlines Inc.,1-1-2015,0.083170,3.523810
2,AA,American Airlines Inc.,1-10-2014,0.062469,3.090909
3,AA,American Airlines Inc.,1-11-2014,0.108637,4.750000
4,AA,American Airlines Inc.,1-12-2013,0.088462,4.344828


In [ ]:
monthly.to_csv('monthly-ratings-final.csv', index = False)

In [ ]:
# compute word frequencies and individual polarities for unigram and bigram
airline_list = reviews["Airline_Name"].unique().tolist()
reviews[["month", "year"]] = reviews[["month", "year"]].astype(str)
m_list = reviews["month"].unique().tolist()
y_list = reviews["year"].unique().tolist()

# reviews["month_year"] = reviews[["month", "year"]].apply(lambda x: str(x[0])+"-"+str(x[1]), axis = 1)
# my_list = reviews["month_year"].unique().tolist()

# group by airline, month-year
freq_df = pd.DataFrame(columns = ["Airline_Name", "year", "month", "Word", "Frequency", "Polarity"])

# inefficient - but one-time activity
for a in airline_list:
    for y in y_list:
        for m in m_list:
            df = reviews[(reviews["Airline_Name"] == a) & (reviews["month"] == m) & (reviews["year"] == y)]
            vect = CountVectorizer(stop_words = None, ngram_range = (1, 2), max_features = 50)
            if df.shape[0] != 0:
                X = vect.fit_transform(df["cleaned_text"].values)
                count_vect_df = pd.DataFrame(X.todense(), columns = vect.get_feature_names()).T.sum(axis = 1).reset_index()
                count_vect_df.columns.values[0] = "Word"
                count_vect_df.columns.values[1] = "Frequency"
                count_vect_df.sort('Frequency', ascending = False, inplace = True)
                count_vect_df["Polarity"] = count_vect_df["Word"].apply(lambda x: float(TextBlob(str(x)).sentiment.polarity))
                for i, row in count_vect_df.iterrows():
                    freq_df = freq_df.append({'Airline_Name': a, 'month': m, "year": y, 'Word': row['Word'], 
                                              'Frequency': row['Frequency'], 'Polarity': row['Polarity']}, ignore_index = True)

In [ ]:
freq_df.head()

In [ ]:
freq_df.to_csv('wordcloud_input.csv', index = False)

<h3>Extracting weather data through wunderground API</h3>

https://www.wunderground.com/weather/api/

In [ ]:
# relevant libraries
import csv
import os
import requests

In [ ]:
# free API key for access to data
api_key = "1df0273b996bac82"

# coordinates for most common departure airport = "Hartsfield Jackson Atlanta International Airport" = "ATL"
# data["Departure_Airport_Name].value_counts()
# can only make 500 API calls a day - will only extract data for this airport
coordinates = [("33.63", "-84.42", "ATL")]

# function to extract weather
# insert list of tuples
# time_from/time_to as datetime.date(YYYY,MM,DD)
def get_weather(coords, time_from, time_to):
    for c in coords:
        lat = c[0]
        lon = c[1]
        airport = c[2]
        with open('%s.csv' % airport, 'wb') as out:
            writer = csv.writer(out)
            headers = ["airport", "day", "month", "year", "mean temp", "mean wind speed", "rain", "snow", 
                       "thunder", "tornado"]
            writer.writerow(headers)
            
            start_date = time_from
            end_date = time_to
            
            date = start_date
            while date <= end_date:
                # format as YYYYMMDD
                date_string = date.strftime("%Y%m%d")
                
                # generate url
                url = ("http://api.wunderground.com/api/%s/daily/history_%s/q/%s,%s.json" % (api_key, date_string, lat, lon))
                
                # make request and parse json
                data = requests.get(url).json()
                
                # extract relevant fields and write to output
                for history in data['history']['dailysummary']:
                    row = []
                    row.append(airport)
                    row.append(str(history['date']['mday']))
                    row.append(str(history['date']['mon']))
                    row.append(str(history['date']['year']))
                    row.append(str(history['meantempi'])) # temp in F
                    row.append(str(history['meanwindspdi'])) # wind in mph
                    row.append(str(history['rain']))
                    row.append(str(history['snow']))
                    row.append(str(history['thunder']))
                    row.append(str(history['tornado']))
                    
                    writer.writerow(row)
                date += datetime.timedelta(days = 1)

In [ ]:
# retrieve data for 2008
get_weather(coordinates, datetime.date(2008, 1, 1), datetime.date(2008, 12, 31))